## 3.0 LLMs and Chat Models

In [1]:
from dotenv import load_dotenv
import os

# .env 파일 로드
load_dotenv()

from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

# # OpenAI API 키 확인 (디버깅용)
# print(os.getenv("OPENAI_API_KEY"))

llm = OpenAI(model="gpt-3.5-turbo-instruct")
chat = ChatOpenAI()

a = llm.predict('How many planets are there?')
b = chat.predict('How many planets are there?')

a,b

('\n\nThere are currently eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune. However, there may be more planets beyond our solar system that have yet to be discovered.',
 'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.')

## 3.1 Predict Messages

In [2]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.1)

In [3]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content = 'You are a geography expert. And you only reply on {language}'), 
    AIMessage(
        content = 'Ciao, mi chiamo {name}!'), 
    HumanMessage(
        content = 'What is the distance between {country_a} and {country_b}. Also, what is your name?'
    )]

chat.predict_messages(messages)

AIMessage(content='Mi dispiace, non posso fornire informazioni sulla distanza tra due paesi specifici. Il mio nome è Assistente di Lingua. Come posso aiutarti oggi?')

## 3.2 Prompt Templates

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    'What is the distance between {country_a} and {country_b}'
    )

prompt = template.format(country_a = 'Mexico', country_b = 'Thailand')

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [5]:
template = ChatPromptTemplate.from_messages([
    ('system','You are a geography expert. And you only reply on {language}'),
    ('ai', 'Ciao, mi chiamo {name}!'),
    ('human', 'What is the distance between {country_a} and {country_b}. Also, what is your name?')
    ])

prompt = template.format_messages(
    language = 'Greek',
    name = 'Socrates',
    country_a = 'Mexico',
    country_b = 'Thailand'
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Το χιλιόμετρο μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης.')

## 3.3 OutputParser and LCEL

In [6]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    
    def parse(self, text):
        items = text.strip().split(',')
        return list(map(str.strip, items))
    
p = CommaOutputParser()

p.parse('Hello,How,are,you?')


['Hello', 'How', 'are', 'you?']

In [7]:
template = ChatPromptTemplate.from_messages([
    ('system', 
     "You are a list generating machine. Everything you are asked will be answered with a comma seperated list of max {max_items}. Do not reply with anything else."),
    ('human', '{question}'),
    ]
)

prompt = template.format_messages(
    max_items = 10, 
    question = 'what are the planets?'
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']

In [8]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    'max_items' : 5,
    'question' : 'what are the pokemons?'
})

['Pikachu', 'Charizard', 'Bulbasaur', 'Squirtle', 'Jigglypuff']

## 3.4 Chaining Chains

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks= [StreamingStdOutCallbackHandler()]
    )

chef_prompt = ChatPromptTemplate.from_messages([
    ('system','You are a world-class international chef. You create easty to follow recipes for any types of cuisine with easy to find ingredients.'),
    ('human', 'I want to cook {cuisine} food.'),
])

chef_chain = chef_prompt | chat

In [10]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {'recipe' : chef_chain} | veg_chain

final_chain.invoke({
    'cuisine' : 'indian',
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Chicken Tikka Masala

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tbsp lemon juice
- 2 tbsp vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) diced tomatoes
- 2 tbsp tomato paste
- 1 tsp ground cumin
- 1 tsp ground coriander
- 1 tsp paprika
- 1 tsp garam masala
- 1/2 tsp turmeric
- Salt and pepper, to taste
- Fresh cilantro, for garnish
- Cooked rice or naan bread, for serving

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tbsp vegetable oil, half of the minced garlic, half of the grated ginger, and a pinch of salt. Add the chicken pieces and marinate for at least 1 hour, preferably overnight.

2. Preheat the oven to 400°F (200°C). Thread the marinat

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare these alternatives:\n\n1. Tofu: Use firm or extra-firm tofu for this recipe. Drain the tofu and pat it dry with paper towels to remove excess moisture. Cut the tofu into bite-sized cubes and follow the same marinating process as you would with the chicken. Tofu absorbs flavors well, so marinating it for a longer time will enhance the taste. You can bake or pan-fry the marinated tofu until it is golden and slightly crispy before adding it to the masala sauce.\n\n2. Paneer: Paneer is a type of Indian cheese that holds its shape well when cooked. You can either make your own paneer at home or buy it from the store. Cut the paneer into cubes and lightly fry them in a pan until they develop a golden crust. You can then add the paneer to the masala sauce towards the end of the cooking process to prevent it from